In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tarfile
import math
from scipy import stats
import statsmodels.formula.api as smf

In [2]:
filename = "MovieSummaries.tar.gz"
ds = tarfile.open(filename)
print(ds.getnames())
tf = tarfile.open(filename)
tf.extractall('MS_decomp')

['MovieSummaries', 'MovieSummaries/tvtropes.clusters.txt', 'MovieSummaries/name.clusters.txt', 'MovieSummaries/plot_summaries.txt', 'MovieSummaries/README.txt', 'MovieSummaries/movie.metadata.tsv', 'MovieSummaries/character.metadata.tsv']


In [3]:
movie = pd.read_csv('./MS_decomp/MovieSummaries/'+'movie.metadata.tsv', 
                 sep='\t',
                 names=['Wikipedia_movie_ID',
                        'Freebase_movie_ID',
                        'Movie_name',
                        'Movie_release_date',
                        'Movie_box_office_revenue',
                        'Movie_runtime',
                        'Movie_languages',
                        'Movie_countries',
                        'Movie_genres'],
                 header=None)

In [4]:
character = pd.read_csv('./MS_decomp/MovieSummaries/'+'character.metadata.tsv', 
                 sep='\t',
                 names=['Wikipedia_movie_ID',
                        'Freebase_movie_ID',
                        'Movie_release_date',
                        'Character_name',
                        'Actor_date_of_birth',
                        'Actor_gender',
                        'Actor_height',
                        'Actor_ethnicity',
                        'Actor_name',
                        'Actor_age_at_movie_release',
                        'Freebase_character/actor_map_ID',
                        'Freebase_character_ID',
                        'Freebase_actor_ID'],
                 header=None)

In [5]:
plots = pd.read_csv('./MS_decomp/MovieSummaries/'+'plot_summaries.txt', 
                 sep='\t',
                 names=['Wikipedia_movie_ID',
                        'Summary'],
                 header=None)

In [6]:
movie.head(3)

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."


In [7]:
character.head(3)

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_release_date,Character_name,Actor_date_of_birth,Actor_gender,Actor_height,Actor_ethnicity,Actor_name,Actor_age_at_movie_release,Freebase_character/actor_map_ID,Freebase_character_ID,Freebase_actor_ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l


In [8]:
plots.head(3)

,Wikipedia_movie_ID,Summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...


# Data cleaning

In [9]:
#move release dates and actor birth dates to datetime format and keep only release year
character.Movie_release_date = pd.to_datetime(character['Movie_release_date'],
                                              errors='coerce').dt.year

movie.Movie_release_date = pd.to_datetime(movie['Movie_release_date'],
                                              errors='coerce').dt.year

In [10]:
#sort dataframes by ascending release year
character.sort_values(by=['Movie_release_date'], ascending=True, inplace=True)

movie.sort_values(by=['Movie_release_date'], ascending=True, inplace=True)

In [11]:
#merge dataframes
ds = pd.merge(movie,character)

In [12]:
#find movies without box office and filter them out
bad_ids = ds.loc[ds['Movie_box_office_revenue'].isnull()].index
ds_filt = ds.drop(bad_ids)

In [13]:
#sort dataframe by movie release date
#ds_filt.sort_values(by=['Movie_release_date'], ascending=True, inplace=True)

In [14]:
len(ds_filt)

101028

In [24]:
ds_filt.groupby('Actor_name').count()

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,Character_name,Actor_date_of_birth,Actor_gender,Actor_height,Actor_ethnicity,Actor_age_at_movie_release,Freebase_character/actor_map_ID,Freebase_character_ID,Freebase_actor_ID
Actor_name,,,,,,,,,,,,,,,,,,
'Squeeks' the Caterpillar,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,1,1
40 Glocc,1,1,1,1,1,1,1,1,1,0,1,1,0,0,1,1,0,1
50 Cent,6,6,6,6,6,6,6,6,6,4,6,6,6,6,6,6,4,6
A Martinez,2,2,2,2,2,2,2,2,2,1,2,2,0,2,2,2,1,2
A. Delon Ellis Jr.,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Şener Şen,1,1,1,0,1,1,1,1,1,1,1,1,0,0,0,1,1,1
Şevval Sam,1,1,1,1,1,1,1,1,1,0,1,0,0,0,1,1,0,1
Željko Ivanek,17,17,17,17,17,17,17,17,17,15,17,17,17,17,17,17,15,17


In [19]:
ds_filt.head(40)

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,Character_name,Actor_date_of_birth,Actor_gender,Actor_height,Actor_ethnicity,Actor_name,Actor_age_at_movie_release,Freebase_character/actor_map_ID,Freebase_character_ID,Freebase_actor_ID
23,28703057,/m/0czdh_n,The Corbett-Fitzsimmons Fight,1897.0,100000.0,NaN,{},{},"{""/m/01z02hx"": ""Sports""}",NaN,1866-09-01,M,NaN,NaN,James J. Corbett,NaN,/m/0gvztx7,NaN,/m/037fcq
1663,1762869,/m/05tz4w,The Squaw Man,1914.0,244700.0,74.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0hfjk"": ""Weste...",NaN,1874-05-27,M,NaN,NaN,Dustin Farnum,NaN,/m/0k6jrc,NaN,/m/06x_50
1746,15340505,/m/03m5kln,Neptune's Daughter,1914.0,1000000.0,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi...",NaN,1887-07-06,F,NaN,NaN,Annette Kellerman,NaN,/m/040nkbv,NaN,/m/047c4t
2267,15130915,/m/03hjhh9,The Million Dollar Mystery,1914.0,1500000.0,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi...",NaN,1888-04-27,F,NaN,NaN,Florence La Badie,NaN,/m/040n878,NaN,/m/01969m
2268,15130915,/m/03hjhh9,The Million Dollar Mystery,1914.0,1500000.0,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi...",NaN,1884-03-27,M,NaN,NaN,James Cruze,NaN,/m/040n87f,NaN,/m/0bzgb4
2269,15130915,/m/03hjhh9,The Million Dollar Mystery,1914.0,1500000.0,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi...",NaN,1889-09-09,F,NaN,NaN,Marguerite Snow,NaN,/m/040n87l,NaN,/m/0krysx
2270,15130915,/m/03hjhh9,The Million Dollar Mystery,1914.0,1500000.0,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi...",NaN,1873-07-08,M,NaN,NaN,Frank Farrington,NaN,/m/040n87r,NaN,/m/0ksr48
2271,15130915,/m/03hjhh9,The Million Dollar Mystery,1914.0,1500000.0,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi...",NaN,NaN,F,NaN,NaN,Lila Chester,NaN,/m/09j0tyb,NaN,/m/04f3n2j
2272,15130915,/m/03hjhh9,The Million Dollar Mystery,1914.0,1500000.0,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi...",NaN,1877-12-18,M,NaN,NaN,Sidney Bracey,NaN,/m/04dd_3w,NaN,/m/02z7tbt
2659,14613957,/m/03gqmqv,Rose of the Rancho,1914.0,87028.0,NaN,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/0hfjk"": ""Western"", ""/m/06ppq"": ""Silent fi...",NaN,1884-09-30,F,NaN,NaN,Bessie Barriscale,NaN,/m/040mfpq,NaN,/m/0c1g1d


## Preliminary results on the impact of one actor

In [ ]:
#slice the character dataframe to get only Leonardo Di Caprio movies
nc = pd.DataFrame(ds_filt.query("Actor_name == 'Leonardo DiCaprio'")).reset_index(drop=True)

In [ ]:
#number of movies Leonardo Di Caprio starred in
print('Leonardo Di Caprio starred in ' + str(len(nc)) + ' movies')

In [ ]:
nc.head(3)

As a preliminary analysis, we plot the distribution of box office revenues for Leonardo Di Caprio movies to see the type of distribution. From the histogram below, we observe that most of the movies have a low box office revenue (skewed distribution); the goal is to verify if there is a correlation between the 'experience' of the actor (number of times he has starred in the movie) and the revenue. 

From the box plot below it is possible to distinguish the median and the quartiles of the distribution. A couple of outliers make it difficult to visualize the distribution properly. To correctly read the data, we proceed to remove them.

In [ ]:
#plot distribution
fig, ax = plt.subplots(1,2, figsize=(8,3), sharey = True)

sbplt = ax[0]
sbplt.hist(nc.Movie_box_office_revenue, bins=15, orientation='horizontal')
sbplt.set_title('Histogram')

sbplt = ax[1]
sbplt.boxplot(nc.Movie_box_office_revenue)
sbplt.set_title('Box plot')
    
fig.tight_layout()
fig.text(0,0.3, "Box office revenue", rotation = 90)
plt.show()

In [ ]:
#remove outliers
nc.query('Movie_box_office_revenue < 0.5e9', inplace=True)
nc.reset_index(inplace=True, drop=True)
print('The new number of movies considered is ' + str(len(nc)))

In this case, the distribution is way more readable. Most of the movies are in the low box office part

In [ ]:
#some stats
print('The median is ' + str(np.median(nc.Movie_box_office_revenue)/1e6) + ' M$')
print('The first quartile is ' + str(np.quantile(nc.Movie_box_office_revenue,0.25)/1e6) + ' M$')
print('The third quartile is ' + str(np.quantile(nc.Movie_box_office_revenue,0.75)/1e6) + ' M$')

In [ ]:
#plot again

fig, ax = plt.subplots(1,2, figsize=(8,3), sharey = True)

sbplt = ax[0]
sbplt.hist(nc.Movie_box_office_revenue, bins=10, orientation='horizontal')
sbplt.set_title('Histogram')

sbplt = ax[1]
sbplt.boxplot(nc.Movie_box_office_revenue)
sbplt.set_title('Box plot')
    
fig.tight_layout()
fig.text(0,0.3, "Box office revenue", rotation = 90)
plt.show()

### Actor's experience

In [ ]:
#series with number of times the actor starred before (related to dataframe)
#in reality there are more movies
exp = np.zeros(len(nc)) 
t = 1
for i in range(len(nc)):
    exp[i] = t
    t += 1

In [ ]:
nc['experience'] = exp #add to dataframe

In [ ]:
#is there a correlation between experience and box office revenue??
stats.pearsonr(nc['experience'],nc['Movie_box_office_revenue']) #pearson correlation

In [ ]:
stats.spearmanr(nc['experience'],nc['Movie_box_office_revenue'])

Applying the Pearson and Spearman relation, we obtain a positive correlation of **0.59** and **0.63**, respectively. This seems to lead to a link between experience and box office revenue.

In [ ]:
#plot linear correlation
sns.lmplot(x='experience',y='Movie_box_office_revenue', data=nc)
plt.show()

### Regression analysis

To understand a bit better the influence of the other parameters on the box office revenue, a model of the revenue based on multiple factors can be tested. The factors taken into consideration are:
- Movie runtime;
- Experience.

In [ ]:
#copy dataframe
nc_reg = nc.copy()

In [ ]:
#logarithmic scaling of continuous variables
nc_reg['Movie_box_office_revenue'] = np.log(nc_reg['Movie_box_office_revenue'])
#nc_reg['Movie_runtime'] = np.log(nc_reg['Movie_runtime'])
#nc_reg['Movie_release_date'] = np.log(nc_reg['Movie_release_date'])
#nc_reg['experience'] = np.log(nc_reg['experience'])

In [ ]:
#standardize continuous variables
#nc_reg['Movie_box_office_revenue'] = (nc_reg['Movie_box_office_revenue'] - nc_reg['Movie_box_office_revenue'].mean())/nc_reg['Movie_box_office_revenue'].std()
#nc['Movie_release_date'] = (nc['Movie_release_date'] - nc['Movie_release_date'].mean())/nc['Movie_release_date'].std()
#nc['Actor_age_at_movie_release'] = (nc['Actor_age_at_movie_release'] - nc['Actor_age_at_movie_release'].mean())/nc['Actor_age_at_movie_release'].std()
#nc['experience'] = (nc['experience'] - nc['experience'].mean())/nc['experience'].std()

In [ ]:
mod = smf.ols(formula='Movie_box_office_revenue ~  Movie_runtime +  experience', data=nc_reg)

res = mod.fit()
print(res.summary())

From the results, it is possible to get interesting information. The R squared is **0.52**, meaning more than half of the variance is explained. The movie runtime (**C=0.03**) and the experience of the actor (**C=0.174**) positively influence the revenue, even if only the latter is statistically significant (**P < 5%**).
A confounder that could naively lead us to overestimate the statistical significance of the actor's experience is inflation. Indeed, over the years the value of money changes; for a proper evaluation, an inflation correction has to be done.

In [ ]:
#take inflation into account
#https://liberalarts.oregonstate.edu/spp/polisci/faculty-staff/robert-sahr/inflation-conversion-factors-years-1774-estimated-2024-dollars-recent-years/individual-year-conversion-factor-table-0
cf = pd.read_csv('inflation_correction.csv')
cf.query('Year > 1969', inplace=True)
cf.reset_index(drop=True, inplace=True)
#cf['Year'] = pd.to_datetime(pd.Series(cf.Year)).dt.year #convert to datetime format

In [ ]:
cf.head(3)

In [ ]:
#function to compute inflation for a certain year
def infl(year):
    inf = cf['CF'][cf['Year'] == year]
    return inf

In [ ]:
#compute inflation for dataframe rows and add column
infl_corr = np.zeros(len(nc))
for i in range(len(nc)):
    infl_corr[i] = infl(nc['Movie_release_date'][i])
nc['CF'] = infl_corr

In [ ]:
#add inflation corrected box office revenue
nc['inflation_revenue'] = nc['Movie_box_office_revenue']/nc['CF']

In [ ]:
nc.head(3)

In [ ]:
#compare the new revenues
plt.plot(nc.Movie_release_date, nc.inflation_revenue/1e6, label='Inflation correction')
plt.plot(nc.Movie_release_date, nc.Movie_box_office_revenue/1e6, label='Real revenue')
plt.legend()
plt.xlabel('Year')
plt.ylabel('Boc office revenue [M$]')
plt.show()

In [ ]:
#run the least mean squares again
nc_reg2 = nc.copy()
#logarithmic scaling of continuous variables
nc_reg2['inflation_revenue'] = np.log(nc_reg2['inflation_revenue'])
#nc_reg2['Movie_runtime'] = np.log(nc_reg2['Movie_runtime'])
#nc_reg2['Movie_release_date'] = np.log(nc_reg2['Movie_release_date'])
#nc_reg2['experience'] = np.log(nc_reg2['experience'])

In [ ]:
mod = smf.ols(formula='inflation_revenue ~ Movie_runtime + experience', data=nc_reg2)

res = mod.fit()
print(res.summary())

From the new results above, it is possible to confirm the statistical significance of the experience on the box office (**P < 5%**), even if its influence has decreased (**C=0.148**), due to inflation.

We can go further in the analysis, by investigating the gender equality/inequality in the movies to see if they are related to revenue as well. The eventual goal would be to establish the benefit of choosing an actor over another one, and so a gender study is fundamental for this purpose.

To have some preliminary results on a single sample, the analysis will regard one of the previously studied Leonardo Di Caprio movies.

In [ ]:
ds.iloc[246888]

In [ ]:
ds.iloc[246887]

In [ ]:
nc_movies = pd.DataFrame([ds_filt['Movie_name'][ds_filt['Actor_name'] == 'Leonardo DiCaprio']]).T.reset_index(drop=True)

In [ ]:
nc_movies

In [ ]:
df_filt[df_filt['Movie_name'] == i for i in nc_movies[1,:]]

In [ ]:
n.groupby('date_transformed').apply(lambda x: pd.Series({'ctr1': x['ctr1'].values })) 

In [ ]:
for i in nc_movies:
    np.concatenate(ds_filt.groupby('Movie_name').get_group(i)['Actor_name'])

In [ ]:
ds_filt.groupby('Movie_name').filter(lambda g: g['Actor_name'] == 'Leonardo Di Caprio')

In [ ]:
ds_filt

In [ ]:
dt = pd.DataFrame(ds.groupby('Movie_name')['Actor_name'].count())

In [ ]:
dt.head(50)